In [1]:
import os

In [2]:
%pwd

'/home/ahmed/project/ML_Chatbot'

In [3]:
# ! pip install unstructured


In [1]:

from langchain_groq import ChatGroq
from dotenv import load_dotenv

import os

load_dotenv()
groq_api_key = os.getenv('GROQ_API_KEY')

llm = ChatGroq(
    model="llama-3.1-8b-instant",
    api_key=groq_api_key,
    max_tokens=500
)

/home/ahmed/miniconda3/envs/chatapp/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
llm.invoke("Hello, LangChain with Groq!")

In [5]:
from langchain_community.document_loaders import PyPDFLoader,DirectoryLoader

loader = PyPDFLoader('project/data/MachineLearning.pdf')
docs = loader.load()

In [6]:
docs

[Document(metadata={'producer': '3-Heights(TM) PDF Optimization Shell 4.8.25.2 (http://www.pdf-tools.com)', 'creator': 'PyPDF', 'creationdate': '2018-12-18T05:07:46+00:00', 'moddate': '2019-01-22T19:51:34+00:00', 'source': 'project/data/MachineLearning.pdf', 'total_pages': 152, 'page': 0, 'page_label': '1'}, page_content='The\nHundred-\nPage\nMachine\nLearning\nBook\nAndriy Burkov'),
 Document(metadata={'producer': '3-Heights(TM) PDF Optimization Shell 4.8.25.2 (http://www.pdf-tools.com)', 'creator': 'PyPDF', 'creationdate': '2018-12-18T05:07:46+00:00', 'moddate': '2019-01-22T19:51:34+00:00', 'source': 'project/data/MachineLearning.pdf', 'total_pages': 152, 'page': 1, 'page_label': '2'}, page_content='“All models are wrong, but some are useful.”\n— George Box\nThe book is distributed on the “read ﬁrst, buy later” principle.\nAndriy Burkov The Hundred-Page Machine Learning Book - Draft'),
 Document(metadata={'producer': '3-Heights(TM) PDF Optimization Shell 4.8.25.2 (http://www.pdf-tool

In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_spliter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

text_spliter

In [8]:
from langchain_core.documents import Document

all_chunks = []

for doc in docs:
    chunks = text_spliter.split_text(doc.page_content)

    for i,chunk in enumerate(chunks):
        chunk_doc = Document(
            page_content= chunk,
            metadata = {'source':doc.metadata['source'],'chunk':i}
        )
        all_chunks.append(chunk_doc)

In [ ]:
all_chunks

In [10]:
print(f"Created {len(all_chunks)} chunks")


Created 404 chunks


In [11]:
from langchain_community.embeddings import HuggingFaceEmbeddings

model_name="sentence-transformers/all-MiniLM-l6-v2"
embedding = HuggingFaceEmbeddings(model_name=model_name)
embedding

/tmp/ipykernel_8519/2373350834.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name=model_name)


HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-l6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [12]:
from langchain_community.vectorstores import FAISS
vector_store = FAISS.from_documents(all_chunks,embedding)

In [18]:
vector_store.save_local("ml_faiss_index")

#ret = FAISS.load_local("ml_faiss_index/index.pkl",embedding)

In [13]:
retriver = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 1})

In [19]:
query = "Support Vector Machine?"

docs_and_scores = retriver.vectorstore.similarity_search_with_score(query, k=4)

for doc, score in docs_and_scores:
    print("--------")
    print("Score:", score)
    print("Metadata:", doc.metadata)
    print(doc.page_content[:400])

--------
Score: 0.93872815
Metadata: {'source': 'project/data/MachineLearning.pdf', 'chunk': 3}
or regression, but not both.
Usually, each library provides the documentation that explains what kind of problem each
algorithm solves, what input values are allowed and what kind of output the model produces.
The documentation also provides information on hyperparameters.
3If it’s really necessary, the score for SVM and kNN predictions could be synthetically created using some
simple techniques.

--------
Score: 0.9656693
Metadata: {'source': 'project/data/MachineLearning.pdf', 'chunk': 1}
example to the training set, rebuild the model and continue the process until some stopping
criterion is satisﬁed. A stopping criterion can be chosen in advance (the maximum number
of requests to the expert based on the available budget) or depend on how well our model
performs according to some metric.
The support vector-based active learning strategy consists in building an SVM model using
the l
-------

In [49]:
from langchain_core.prompts import PromptTemplate

prompt_template = """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, say that you don't know.
Use three sentences maximum and keep the answer concise.

Question: {input}
Context: {context}
Answer:
"""

prompt = PromptTemplate.from_template(prompt_template)

print(prompt)


input_variables=['context', 'input'] input_types={} partial_variables={} template="You are an assistant for question-answering tasks.\nUse the following pieces of retrieved context to answer the question.\nIf you don't know the answer, say that you don't know.\nUse three sentences maximum and keep the answer concise.\n\nQuestion: {input}\nContext: {context}\nAnswer:\n"


In [21]:
def get_context(question):
    docs = retriver.invoke(question)

    return "\n\n".join(doc.page_content for doc in docs)
context = get_context("whats SVM?")
context

'8 Advanced Practice\nThis chapter contains the description of techniques that you could ﬁnd useful in your practice\nin some contexts. It’s called “Advanced Practice” not because the presented techniques are\nmore complex, but rather because they are applied in some very speciﬁc contexts. In many\npractical situations, you will most likely not need to resort to using these techniques, but\nsometimes they are very helpful.\n8.1 Handling Imbalanced Datasets\nIn many practical situations, your labeled dataset will have underrepresented the examples\nof some class. This is the case, for example, when your classiﬁer has to distinguish between\ngenuine and fraudulent e-commerce transactions: the examples of genuine transactions are\nmuch more frequent. If you use SVM with soft margin, you can deﬁne a cost for misclassiﬁed\nexamples. Because noise is always present in the training data, there are high chances that\nmany examples of genuine transactions would end up on the wrong side of the d

In [25]:
from langchain_core.runnables import RunnableParallel,RunnablePassthrough,RunnableLambda

runable = (RunnableParallel(
    {'context':RunnableLambda(get_context),
    'question':RunnablePassthrough()
    }
))


chain = runable | prompt | llm

response = chain.invoke('whats SVM?')
response.content

'SVM stands for Support Vector Machine, a classification technique used in machine learning. It can handle imbalanced datasets by allowing a soft margin, where a cost can be defined for misclassified examples. This helps to avoid noise in the training data and improve accuracy.'

In [53]:
response

{'input': 'whats SVM?',
 'context': [Document(id='8a82739e-ad5c-4591-b966-f1dad3be7e06', metadata={'source': 'project/data/MachineLearning.pdf', 'chunk': 0}, page_content='8 Advanced Practice\nThis chapter contains the description of techniques that you could ﬁnd useful in your practice\nin some contexts. It’s called “Advanced Practice” not because the presented techniques are\nmore complex, but rather because they are applied in some very speciﬁc contexts. In many\npractical situations, you will most likely not need to resort to using these techniques, but\nsometimes they are very helpful.\n8.1 Handling Imbalanced Datasets\nIn many practical situations, your labeled dataset will have underrepresented the examples\nof some class. This is the case, for example, when your classiﬁer has to distinguish between\ngenuine and fraudulent e-commerce transactions: the examples of genuine transactions are\nmuch more frequent. If you use SVM with soft margin, you can deﬁne a cost for misclassiﬁed\

In [26]:
chain

{
  context: RunnableLambda(get_context),
  question: RunnablePassthrough()
}
| PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks.\nUse the following pieces of retrieved context to answer the question.\nIf you don't know the answer, say that you don't know.\nUse three sentences maximum and keep the answer concise.\n\nQuestion: {question}\nContext: {context}\nAnswer:\n")
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7981298fee40>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x79812961fec0>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'), max_tokens=500)

In [ ]:
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_classic.chains.retrieval import create_retrieval_chain



combine_docs_chain = create_stuff_documents_chain(llm, prompt)

result = create_retrieval_chain(retriver, combine_docs_chain)
result

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7980d5ac9640>, search_kwargs={'k': 1}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks.\nUse the following pieces of retrieved context to answer the question.\nIf you don't know the answer, say that you don't know.\nUse three sentences maximum and keep the answer concise.\n\nQuestion: {input}\nConte

In [51]:
print(result.input_schema.schema())


{'properties': {'input': {'title': 'Input'}}, 'required': ['input'], 'title': 'RunnableParallel<context>Input', 'type': 'object'}


/tmp/ipykernel_8519/2134537224.py:1: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  print(result.input_schema.schema())


In [52]:
response = result.invoke({
    "input": "whats SVM?"
})

print(response["answer"])


SVM stands for Support Vector Machine, a machine learning technique used for classification and regression tasks. It's mentioned in the context of handling imbalanced datasets, particularly when dealing with genuine and fraudulent transactions. SVM with a soft margin allows for the definition of a cost for misclassified examples.
